In [1]:
import numpy as np
import pandas as pd
import datetime
import re
import requests
from bs4 import BeautifulSoup
import warnings

In [2]:
def prepare_data():
    # Newyork:
    # https://raw.githubusercontent.com/nychealth/coronavirus-data/master/totals/data-by-modzcta.csv
    # City of San Antonio:
    # https://cosacovid-cosagis.hub.arcgis.com/datasets/CoSAGIS::covid19-deaths-by-zip-code/about
    # Wisconsin:
    # https://data.dhsgis.wi.gov/datasets/wi-dhs::covid-19-data-by-zip-code-tabulation-area-v2/about
    ny_df = pd.read_csv('Data/Covid Data/newyork-covid19-cases-and-deaths.csv')
    ny_df = ny_df[['MODIFIED_ZCTA', 'COVID_CASE_COUNT', 'COVID_DEATH_COUNT']]
    ny_df.columns = ['Zipcode', 'Case Counts', 'Death Counts']
    display(ny_df)
    sa_df = pd.read_csv('Data/Covid Data/san-antonio-covid19-cases-and-deaths.csv')
    sa_df = sa_df[['ZIP_CODE', 'Positive','Deaths']]
    sa_df.columns = ['Zipcode', 'Case Counts', 'Death Counts']
    display(sa_df)
    wi_df = pd.read_csv('Data/Covid Data/wisconsin-covid19-cases-and-deaths.csv')
    wi_df = wi_df[['GEOID', 'POS_CUM_CP', 'DTH_CUM_CP']]
    wi_df.columns = ['Zipcode', 'Case Counts', 'Death Counts']
    display(wi_df)
    frames = [ny_df, sa_df, wi_df]
    df = pd.concat(frames)
    return df

In [3]:
df = prepare_data()

,Zipcode,Case Counts,Death Counts
0,10001,7745,57
1,10002,19171,412
2,10003,13634,84
3,10004,973,2
4,10005,2323,1
...,...,...,...
172,11691,19207,614
173,11692,5068,154
174,11693,3256,63
175,11694,5783,133


,Zipcode,Case Counts,Death Counts
0,78002,2712,21
1,78015,2407,11
2,78023,5907,41
3,78069,405,8
4,78073,3006,18
...,...,...,...
61,78259,5505,36
62,78260,7417,29
63,78261,5200,14
64,78263,1305,7


,Zipcode,Case Counts,Death Counts
0,54403,6922,90
1,54405,935,24
2,54406,932,7
3,54407,329,1
4,54408,252,3
...,...,...,...
770,54540,186,1
771,54541,391,13
772,54542,82,1
773,54545,165,6


In [4]:
def scrape(z):
        z = str(z)
        source = requests.get('https://censusreporter.org/profiles/86000US{zip}-{zip}/'.format(zip=z)).text
        soup = BeautifulSoup(source, 'lxml')
        s = soup.findAll('script',type="text/javascript")[1]
        return str(s)

def parse(param, sp):
    param = '"'+param+'", '
    try:
        var = float(re.findall(param+'"values": {"this":\s*([+-]?[0-9]+\.[0-9]+)',sp)[0])
    except:
        var = 0

    return var

def getSocioDem(zipcodes):
    df = pd.DataFrame(columns = [ 'Zipcode', 'Population', 'Median age', 'Under 18(%)', '18 to 64(%)', '65 and over(%)', 'Male(%)', 'Female(%)', 'White(%)', "Black(%)", "Native(%)", "Asian(%)", "Islander(%)", "Two plus(%)", "Hispanic(%)", "Per capita income (USD)", "Median household income (USD)", "Below poverty line(%)",
                                'Mean travel time to work (Minutes)', 'Drove Alone (%)', 'Carpooled (%)', 'Public Transit (%)', 'Bicycle (%)', 'Walked (%)', 'Other (%)', 'Worked at home (%)', 'Number of households', 'Persons per household', 'Married (%)', 'Single (%)',
                                'Number of housing units', 'Occupied housing (%)', 'Vacant housing (%)', 'Owner Occupied (%)', 'Renter Occupied (%)', 'Median housing value',
                                'Moved Since Prev Year(%)', 'Same House Prev Year(%)', 'No Degree(%)', 'High School(%)', 'Some College(%)', "Bachelor's(%)", "Post-grad(%)", 'Foriegn Born Population(%)', 'Europe(%)', 'Asia(%)', 'Africa(%)', 'Oceania(%)', 'Latin America(%)', 'North America(%)'],
                                index = list(range(0,len(zipcodes))))

    i = 0
    for zip in zipcodes:
        try:
            s = scrape(zip)
        except:
            continue
        var = '"full_geoid": "86000US{z}", "total_population":'.format(z=str(zip))
        population = int(re.findall(var+'\s*([+-]?[0-9]+)',s)[0])

        median_age = parse("Median age",s)
        percent_under18 = parse("Under 18",s)
        percent_18to64 = parse("18 to 64",s)
        percent_65andOver = parse("65 and over",s)

        percent_male = parse("Male",s)
        percent_female = parse("Female",s)

        percent_white = parse("White",s)
        percent_black = parse("Black",s)
        percent_native = parse("Native",s)
        percent_asian = parse("Asian",s)
        percent_islander = parse("Islander",s)
        pecent_two_plus = parse("Two\+",s)
        percent_hispanic = parse("Hispanic",s)

        per_capita = parse("Per capita income", s)
        median_household_income = parse("Median household income",s)

        percent_below_poverty = parse("Persons below poverty line",s)

        mean_travel_time = parse("Mean travel time to work",s)
        drove_alone = parse("Drove alone",s)
        carpooled = parse("Carpooled",s)
        public_transit = parse("Public transit",s)
        bicycle = parse("Bicycle",s)
        walked = parse("Walked",s)
        other = parse("Other",s)
        worked_at_home = parse("Worked at home",s)

        number_of_households = parse("Number of households",s)
        persons_per_household = parse("Persons per household",s)

        married = parse("Married",s)
        single = parse("Single",s)

        number_of_housing_units = parse("Number of housing units",s)
        occupied_housing_units = parse("Occupied",s)
        vacant_housing_units = parse("Vacant",s)
        owner_housing_units = parse("Owner occupied",s)
        renter_housing_units = parse("Renter occupied",s)

        median_value_owner_occupied = parse("Median value of owner-occupied housing units",s)

        moved_since_previous_year = parse("Moved since previous year",s)
        same_house_year_ago = parse("Same house year ago",s)

        no_degree = parse("No degree",s)
        high_school = parse("High school",s)
        some_college = parse("Some college",s)
        bachelors = parse("Bachelor's",s)
        post_grad = parse("Post-grad",s)

        foriegn_born_pop = parse("Foreign-born population",s)
        europe = parse('Europe',s)
        asia = parse('Asia',s)
        africa = parse('Africa',s)
        oceania = parse('Oceania',s)
        latin_america = parse('Latin America',s)
        north_america = parse('North America',s)

        df.iloc[i] = [zip, population, median_age, percent_under18, percent_18to64, percent_65andOver, percent_male, percent_female, percent_white, percent_black, percent_native, percent_asian, percent_islander, pecent_two_plus, percent_hispanic, per_capita, median_household_income, percent_below_poverty, mean_travel_time,
        drove_alone, carpooled, public_transit, bicycle, walked, other, worked_at_home, number_of_households, persons_per_household, married, single, number_of_housing_units, occupied_housing_units, vacant_housing_units, owner_housing_units, renter_housing_units,
        median_value_owner_occupied, moved_since_previous_year, same_house_year_ago, no_degree, high_school, some_college, bachelors, post_grad, foriegn_born_pop, europe, asia, africa, oceania, latin_america, north_america]

        i+=1
    return df

In [5]:
#crape social demographic data 
zipcodes = df.Zipcode.tolist()
socio_df = getSocioDem(zipcodes)

In [6]:
socio_df['Zipcode'].isnull().sum()
socio_df.iloc[:-2]

,Zipcode,Population,Median age,Under 18(%),18 to 64(%),65 and over(%),Male(%),Female(%),White(%),Black(%),...,Some College(%),Bachelor's(%),Post-grad(%),Foriegn Born Population(%),Europe(%),Asia(%),Africa(%),Oceania(%),Latin America(%),North America(%)
0,10001,25026,36.1,10.98,76.38,12.64,49.01,50.99,54.51,6.14,...,10.28,43.84,28.37,28.71,19.08,49.55,2.25,3.52,21.46,4.13
1,10002,74363,44.8,13.09,63.18,23.74,49.29,50.71,22.16,7.77,...,12.98,23.86,10.65,41.04,5.55,79.01,0.61,1.27,12.85,0.7
2,10003,54671,31.9,7.18,80.16,12.66,48.78,51.22,67.98,5.01,...,9.23,45.16,36.0,18.84,30.11,41.21,1.84,6.09,15.72,5.03
3,10004,3310,38.4,7.37,84.23,8.4,47.34,52.66,66.89,4.5,...,3.48,45.67,48.33,20.36,27.89,53.26,2.97,7.42,2.08,6.38
4,10005,8664,30.4,10.2,88.73,1.06,47.68,52.32,70.16,2.01,...,1.6,51.52,40.15,17.68,36.36,41.91,0.72,9.01,8.49,3.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,54539,1261,57.9,11.26,51.39,37.35,48.77,51.23,93.66,1.82,...,39.72,15.78,12.11,0.32,100.0,0,0,0,0,0
1012,54540,880,58.2,11.59,60.34,28.07,52.95,47.05,90.68,0,...,37.45,20.71,13.17,2.84,68.0,0,0,0,32.0,0
1013,54541,1392,45.1,20.55,58.84,20.62,44.4,55.6,80.1,4.17,...,29.82,9.58,6.09,1.22,11.76,88.24,0,0,0,0
1014,54542,500,52.9,13.0,56.2,30.8,50.0,50.0,79.2,0,...,32.74,10.49,4.86,0,0,0,0,0,0,0


In [7]:
merge_df = pd.merge(socio_df, df, how='inner', on='Zipcode')
zero_rows = merge_df[merge_df['Population'] == 0].index.tolist()
merge_df = merge_df.drop(zero_rows)
merge_df['Death Counts(Per 1000)'] = (merge_df['Death Counts'] / merge_df['Population']) * 1000
merge_df['Case Counts(Per 1000)'] = (merge_df['Case Counts'] / merge_df['Population']) * 1000
merge_df

,Zipcode,Population,Median age,Under 18(%),18 to 64(%),65 and over(%),Male(%),Female(%),White(%),Black(%),...,Europe(%),Asia(%),Africa(%),Oceania(%),Latin America(%),North America(%),Case Counts,Death Counts,Death Counts(Per 1000),Case Counts(Per 1000)
0,10001,25026,36.1,10.98,76.38,12.64,49.01,50.99,54.51,6.14,...,19.08,49.55,2.25,3.52,21.46,4.13,7745,57,2.277631,309.478143
1,10002,74363,44.8,13.09,63.18,23.74,49.29,50.71,22.16,7.77,...,5.55,79.01,0.61,1.27,12.85,0.7,19171,412,5.54039,257.80294
2,10003,54671,31.9,7.18,80.16,12.66,48.78,51.22,67.98,5.01,...,30.11,41.21,1.84,6.09,15.72,5.03,13634,84,1.536464,249.382671
3,10004,3310,38.4,7.37,84.23,8.4,47.34,52.66,66.89,4.5,...,27.89,53.26,2.97,7.42,2.08,6.38,973,2,0.60423,293.957704
4,10005,8664,30.4,10.2,88.73,1.06,47.68,52.32,70.16,2.01,...,36.36,41.91,0.72,9.01,8.49,3.52,2323,1,0.11542,268.12096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,54540,880,58.2,11.59,60.34,28.07,52.95,47.05,90.68,0,...,68.0,0,0,0,32.0,0,186,1,1.136364,211.363636
1013,54541,1392,45.1,20.55,58.84,20.62,44.4,55.6,80.1,4.17,...,11.76,88.24,0,0,0,0,391,13,9.33908,280.890805
1014,54542,500,52.9,13.0,56.2,30.8,50.0,50.0,79.2,0,...,0,0,0,0,0,0,82,1,2.0,164.0
1015,54545,859,61.1,11.06,50.99,37.95,51.11,48.89,92.67,0,...,100.0,0,0,0,0,0,165,6,6.984866,192.083818


In [8]:
import wrangle2
df = wrangle2.get_data()
df

,Zipcode,Population,Median age,Under 18(%),18 to 64(%),65 and over(%),Male(%),Female(%),White(%),Black(%),...,Europe(%),Asia(%),Africa(%),Oceania(%),Latin America(%),North America(%),Case Counts,Death Counts,Death Counts(Per 1000),Case Counts(Per 1000)
0,10001,25026,36.1,10.98,76.38,12.64,49.01,50.99,54.51,6.14,...,19.08,49.55,2.25,3.52,21.46,4.13,7745,57,2.277631,309.478143
1,10002,74363,44.8,13.09,63.18,23.74,49.29,50.71,22.16,7.77,...,5.55,79.01,0.61,1.27,12.85,0.7,19171,412,5.54039,257.80294
2,10003,54671,31.9,7.18,80.16,12.66,48.78,51.22,67.98,5.01,...,30.11,41.21,1.84,6.09,15.72,5.03,13634,84,1.536464,249.382671
3,10004,3310,38.4,7.37,84.23,8.4,47.34,52.66,66.89,4.5,...,27.89,53.26,2.97,7.42,2.08,6.38,973,2,0.60423,293.957704
4,10005,8664,30.4,10.2,88.73,1.06,47.68,52.32,70.16,2.01,...,36.36,41.91,0.72,9.01,8.49,3.52,2323,1,0.11542,268.12096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,54540,880,58.2,11.59,60.34,28.07,52.95,47.05,90.68,0,...,68.0,0,0,0,32.0,0,186,1,1.136364,211.363636
1013,54541,1392,45.1,20.55,58.84,20.62,44.4,55.6,80.1,4.17,...,11.76,88.24,0,0,0,0,391,13,9.33908,280.890805
1014,54542,500,52.9,13.0,56.2,30.8,50.0,50.0,79.2,0,...,0,0,0,0,0,0,82,1,2.0,164.0
1015,54545,859,61.1,11.06,50.99,37.95,51.11,48.89,92.67,0,...,100.0,0,0,0,0,0,165,6,6.984866,192.083818


In [9]:
df.to_csv('socio-demographic-and-death-counts.csv')